# Set up Pycap LPR to operate with PEST++ for optimization

In [ ]:
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
import pyemu
import matplotlib.pyplot as plt
import os, shutil, platform

In [ ]:
#### Path to excel input file. Note can use absolute or relative path.
pycap_inputs_excel = Path("../../PyCap/PyCap_USGS/pycap-dss2/Notebooks/Inputs/LPR_Prepped.xlsx")

#### PyCap Run Name is what all your outputs will have as a name. 
pycap_run_name = "LPR_Redux"

#### Base directory for runs
parent_run_path = Path("../pycap_runs")

#### depletion potential calculations directory
base_run_path = parent_run_path / "pycap_base"
pest_path = parent_run_path / "pycap_pest"
template_path = pest_path / "pycap_opt_template"
if pest_path.exists():
    shutil.rmtree(pest_path)
template_path.mkdir(parents=True)

#### finally define the scripts directory
script_path = Path("../scripts")


# Parameterization for PEST++ 

In [ ]:
if not template_path.exists():
    template_path.mkdir(parents=True)

#### let's load up the configuration file

In [ ]:
with open(base_run_path / f"{pycap_run_name}.yml", 'r') as ifp:
        indat = yaml.safe_load(ifp)

#### and now parameterize inputs to vary in optimization. This will only be pumping rates for now

First we set up template (`TPL`) files that allow PEST++ to update model input values by name. We do this by reading in the input (`YML`) file and replacing numeric values with updated values being changed by the algorithm.

In [ ]:
# well-by-well pumping rates
well_keys = [i for i in indat.keys() if i.startswith('well_')]
pending_wells = [i for i in well_keys if 'pending' in indat[i]['status']]

pars = list()
parvals = list()

# then again for pumping rate Q
for k in well_keys:
    cpar = f'{k}__q'
    pars.append(cpar)
    parvals.append(indat[k]['Q'])
    indat[k]['Q'] = f'~{cpar:^45}~'

In [ ]:
# save out tpl file
with open(template_path / f"{pycap_run_name}.yml.tpl", 'w') as ofp:
    ofp.write('ptf ~\n')
    documents = yaml.dump(indat, ofp, default_flow_style = False, sort_keys = False)

In [ ]:
# create DataFrame of parameters
pars_df = pd.DataFrame(index = pars, data= {'parval1':parvals})



In [ ]:
pars_df.sample(5)

### Next we need to be able to read in model ouputs to PEST++
Now we write an instruction file (`INS`) that can navigate model output and read it into PEST++

In [ ]:
# make ins file and external forward run file
# set base case depletion observations
basedeplobs = [f"{indat[k]['name']}:bdpl" for k in indat.keys() if 'stream' in k]

# get list of unique stream names used in the run
unique_rivers = list(set([i.split(':')[0] for i in basedeplobs]))

# add in the totals/sums of proposed/existing/combined depletions for each stream
basedeplobs.extend([f'{i}:{j}:bdpl' for i in unique_rivers for j in ['total_proposed','total_existing','total_combined']])

with open(template_path / 'basedeplobs.dat','w') as ofp:
    [ofp.write(i + '\n') for i in basedeplobs]

In [ ]:
basedeplobs

### Now read in the base case observation values for depletion data

In [ ]:
base_data = pd.read_csv(base_run_path/"output" / f'{pycap_run_name}.table_report.base_stream_depletion.csv', index_col=0)
# read in the observation names and make a DataFrame to keep the results in
bdplobs = pd.read_csv(template_path/'basedeplobs.dat', header=None)
bdplobs.columns =['obsname']
bdplobs.index = bdplobs.obsname
bdplobs['obs_values'] = np.nan

# now map the actual output values to the DataFrame
for cob in bdplobs.obsname:
    riv,wel,_ = cob.split(':')
    bdplobs.loc[cob, 'obs_values'] = base_data.loc[wel][riv]

In [ ]:
bdplobs.sample(5)

### We can combine all the outputs into a single dataframe and make the instruction file we'll need to read in the results

In [ ]:
bdplobs['obs_values'].to_csv(template_path / 'allobs.out', sep = ' ', header=None)

with open(template_path / 'allobs.out.ins', 'w') as ofp:
    ofp.write('pif ~\n')
    [ofp.write(f'l1 w !{i}!\n') for i in bdplobs.index]

### Now we need to make a PEST control file to orchestrate everything. Luckily, `pyemu` makes this straightforward now that we have made the `tpl` and `ins` files

In [ ]:
cwd = os.getcwd()
os.chdir(template_path)
pst = pyemu.Pst.from_io_files(*pyemu.utils.parse_dir_for_io_files('.'))
os.chdir(cwd)

In [ ]:
pars = pst.parameter_data

# let's clean up some of the data and add important values

In [ ]:
# name paramter groups according to the type of parameter
pars.loc[pars.parnme.str.endswith("q"), "pargp"] = "pumping"
# set initial values
pars.loc[pars_df.index,'parval1'] = pars_df.parval1

In [ ]:
pars

In [ ]:
del_pump = .2 #(as a fraction)
# set upper and lower bounds on pumping rates as well - initially allow wells to go down to 0 or increase to 1.2x
pars.loc[pars.pargp=="pumping","parlbnd"] = 0 
pars.loc[pars.pargp=="pumping", "parubnd"] = pars.loc[pars.pargp=="pumping", "parval1"]*(1+del_pump)
pars.partrans = 'none'

In [ ]:
pars

### We need to make some definitions for the linear programming optimization algorithm

In [ ]:

pst.pestpp_options["opt_dec_var_groups"] = "pumping"

# we want to maximize pumping, 
pst.add_pi_equation(pars.loc[pars.pargp=='pumping','parnme'], # parameter names to include in the equation
                    pilbl="obj_well",  # the prior information equation name
                    obs_group="greater_than") # note the "greater_" prefix.     

pst.pestpp_options["opt_direction"] = "max"
pst.pestpp_options["opt_objective_function"] = "obj_well"

### now for some constraints

In [ ]:
obs = pst.observation_data
obs.loc[:,'weight'] = 0

In [ ]:
obs

In [ ]:
constraint_names = ["lpr:total_combined:bdpl"]
obs.loc[constraint_names,"weight"] = 1.0 # just non-zero, the value doesn't matter...
# set constraint value
swgw_rhs = 0.0
obs.loc[constraint_names,"obsval"] = obs.loc[constraint_names,"obsval"]*.9 # the constraint right hand side
# assign to new obs group
obs.loc[constraint_names,"obgnme"] = "less_than_depl"

In [ ]:

pst.parameter_groups.loc["pumping","inctyp"] = "absolute"
pst.parameter_groups.loc["pumping","derinc"] = 2.0 #remember these are multipliers!
pst.parameter_groups.loc["pumping","derinclb"] = 2.0

In [ ]:
# setup and save pst file
pst.control_data.noptmax = 0
pst.model_command = [f'python run_pycap_standalone_opt_mou.py']
pst.pestpp_options['par_sigma_range']=6
pst.pestpp_options['ies_num_reals'] = 50

pst.write(str(template_path / 'lpr_opt.pst'), version=2)



In [ ]:
# Copy over directories

MASTER_path = pest_path / "MASTER_opt"
if MASTER_path.exists():
    shutil.rmtree(MASTER_path)
# copy over the binaries into template first so they get distributed


#copy over the correct binary
if "window" in platform.platform().lower():
    shutil.copy2('../../binaries/PESTPP/windows/pestpp-opt.exe', template_path / 'pestpp-opt.exe')
elif "linux" in platform.platform().lower():
    shutil.copy2('../../binaries/PESTPP/linux/pestpp-opt', template_path / 'pestpp-opt')
elif "mac" in platform.platform().lower():
    shutil.copy2('../../binaries/PESTPP/mac/pestpp-opt', template_path / 'pestpp-opt')

# we also need the forward run script
shutil.copy2(script_path / 'run_pycap_standalone_opt_mou.py', 
             template_path / 'run_pycap_standalone_opt_mou.py')

# and we need the base yml file
shutil.copy2(base_run_path / 'LPR_Redux.yml',
              template_path / 'LPR_Redux.yml')

# finally populate the NOPTMAX=0 and MASTER paths with the template_path files
shutil.copytree(template_path, MASTER_path)
os.remove(MASTER_path / 'allobs.out')

In [ ]:

pyemu.os_utils.run("pestpp-opt lpr_opt.pst /e",cwd=MASTER_path)


In [ ]:

pyemu.os_utils.run("pestpp-opt ",cwd=MASTER_path)

### assuming this worked, set the number of optimization 